# Customer Loyalty Feature Engineering

In [2]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import time
from datetime import datetime 
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('./Data/encoded_df.csv')
df = df.sort_values(by='purchase_date')
df.head(3)

,card_id,city_id,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,...,new_merchant_flag,category_1,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_2_NA,category_3_B,category_3_C,category_3_NA
7258152,C_ID_da2090f28e,69,0,623,M_ID_f001319a61,-11,-0.686802,2017-01-01 00:00:08,9,4,...,False,False,False,False,False,False,False,False,False,False
18429662,C_ID_efced389a0,76,1,842,M_ID_18038b5ae7,-12,-0.566590,2017-01-01 00:00:59,2,37,...,False,False,False,True,False,False,False,True,False,False
14876034,C_ID_83561fe74a,233,1,661,M_ID_52d3026407,-13,-0.559227,2017-01-01 00:01:41,9,8,...,False,False,False,False,False,False,False,True,False,False


# Aggregating Boolean Columns

In [5]:
# Shorten column names and isolate boolean features

renaming = {
       'new_merchant_flag': 'new_flag',
       'authorized_flag': 'auth_flag',
       'category_1': 'c1',
       'category_2_2.0': 'c2_2',
       'category_2_3.0': 'c2_3', 
       'category_2_4.0': 'c2_4',
       'category_2_5.0': 'c2_5', 
       'category_2_NA': 'c2_NA', 
       'category_3_B': 'c3_B',
       'category_3_C': 'c3_A', 
       'category_3_NA': 'c3_NA'
}

bool_cols = list(renaming.values())

bool_df = df[['card_id'] + list(renaming.keys())]
bool_df.rename(columns=renaming, inplace=True)
bool_df.head()

,card_id,new_flag,auth_flag,c1,c2_2,c2_3,c2_4,c2_5,c2_NA,c3_B,c3_A,c3_NA
7258152,C_ID_da2090f28e,False,True,False,False,False,False,False,False,False,False,False
18429662,C_ID_efced389a0,False,True,False,False,True,False,False,False,True,False,False
14876034,C_ID_83561fe74a,False,True,False,False,False,False,False,False,True,False,False
28523797,C_ID_479fd6392a,False,True,True,False,False,False,False,True,True,False,False
19914410,C_ID_1cf6056088,False,True,False,False,False,False,False,False,False,False,False


In [43]:
# Defining aggregation functions

def sum0(series):
    sum1 = series.sum()
    sum0 = len(series) - sum1
    return sum0

def quantile25(series):
    return series.quantile(0.25)

def quantile75(series):
    return series.quantile(0.75)

def mode(series):
    return series.mode().iloc[0] if not series.empty else np.nan

def changes(series):
    return (series != series.shift()).sum()

In [89]:
# Aggregating by card_id

bool_features = bool_df.groupby('card_id')[bool_cols].agg([
    'mean',
    'median',
    'sum',
    sum0,
    mode,
    'std',
    'skew',
    changes
])

bool_features.columns = [f"{col}_{agg}" for col, agg in bool_features.columns]
bool_features.reset_index(inplace=True)

bool_features = bool_features.astype({
    col: 'float32' if bool_features[col].dtype == 'float64' else 'int32' if bool_features[col].dtype == 'int64' else bool_features[col].dtype
    for col in bool_features.columns
})

bool_features.head()

,new_flag_mean,new_flag_median,new_flag_sum,new_flag_sum0,new_flag_mode,new_flag_std,new_flag_skew,new_flag_changes,auth_flag_mean,auth_flag_median,...,c3_A_skew,c3_A_changes,c3_NA_mean,c3_NA_median,c3_NA_sum,c3_NA_sum0,c3_NA_mode,c3_NA_std,c3_NA_skew,c3_NA_changes
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,0.013245,0.0,2,149,False,0.114703,8.601160,2,0.768212,1.0,...,1.884419,34,0.000000,0.0,0,151,False,0.000000,0.000000,1
C_ID_0001238066,0.168919,0.0,25,123,False,0.375953,1.785419,2,0.979730,1.0,...,1.209177,51,0.027027,0.0,4,144,False,0.162713,5.893231,9
C_ID_0001506ef0,0.014925,0.0,1,66,False,0.122169,8.185352,2,0.940298,1.0,...,0.000000,1,0.000000,0.0,0,67,False,0.000000,0.000000,1
C_ID_0001793786,0.125506,0.0,31,216,False,0.331965,2.274647,2,0.890688,1.0,...,0.000000,1,0.000000,0.0,0,247,False,0.000000,0.000000,1
C_ID_000183fdda,0.070968,0.0,11,144,False,0.257603,3.374495,2,0.954839,1.0,...,1.078235,56,0.032258,0.0,5,150,False,0.177257,5.346532,8


In [113]:
# Fix one boolean at a time to either 0 or 1 and run statistics on the other booleans

def bool_agg(bool_df, bool_cols):

    merged_features = None
    steps = len(bool_cols)
    start_time = time.time()

    for i, group in enumerate(bool_cols):

        prefix = group.split('_')[0]
        filtered_cols = [value for value in bool_cols if prefix not in value]

        features = bool_df.groupby(['card_id', group])[filtered_cols].agg([
            'mean',
            'median',
            'sum',
            sum0
        ]).reset_index()

        features.columns = ['card_id', group] + [f"{col}_{agg}_{group}" for col, agg in features.columns[2:]]

        features = features.astype({
            col: 'float32' if features[col].dtype == 'float64' 
            else 'int32' if features[col].dtype == 'int64' 
            else features[col].dtype 
            for col in features.columns
        })

        features1 = features[features[group].to_numpy()]
        features0 = features[~features[group].to_numpy()]
        features1.drop(group, axis=1, inplace=True)
        features0.drop(group, axis=1, inplace=True)
        features1.columns = ['card_id'] + [f"{col}_1" for col in features1.columns[1:]]
        features0.columns = ['card_id'] + [f"{col}_0" for col in features0.columns[1:]]

        features = pd.merge(features1, features0, on='card_id', how='outer')

        if merged_features is None:
            merged_features = features
        else:
            merged_features = pd.merge(merged_features, features, on='card_id')

        elapsed_time = time.time() - start_time
        print(f"[{i+1}/{steps}]\tshape:", merged_features.shape, "\ttime:", elapsed_time)

    return merged_features

In [85]:
# Aggregate using above function

merged = bool_agg(bool_df, bool_cols)
merged.head()

[1/11]	shape: (325540, 81) 	time: 165.7909939289093
[2/11]	shape: (325540, 161) 	time: 330.08673906326294
[3/11]	shape: (325540, 241) 	time: 463.56731629371643
[4/11]	shape: (325540, 289) 	time: 531.3272840976715
[5/11]	shape: (325540, 337) 	time: 606.7221009731293
[6/11]	shape: (325540, 385) 	time: 676.7104630470276
[7/11]	shape: (325540, 433) 	time: 754.7787525653839
[8/11]	shape: (325540, 481) 	time: 838.8245973587036
[9/11]	shape: (325540, 545) 	time: 960.0623030662537
[10/11]	shape: (325540, 609) 	time: 1074.687548160553
[11/11]	shape: (325540, 673) 	time: 1175.1983017921448


,card_id,auth_flag_mean_new_flag_1,auth_flag_median_new_flag_1,auth_flag_sum_new_flag_1,auth_flag_sum0_new_flag_1,c1_mean_new_flag_1,c1_median_new_flag_1,c1_sum_new_flag_1,c1_sum0_new_flag_1,c2_2_mean_new_flag_1,...,c2_4_sum_c3_NA_0,c2_4_sum0_c3_NA_0,c2_5_mean_c3_NA_0,c2_5_median_c3_NA_0,c2_5_sum_c3_NA_0,c2_5_sum0_c3_NA_0,c2_NA_mean_c3_NA_0,c2_NA_median_c3_NA_0,c2_NA_sum_c3_NA_0,c2_NA_sum0_c3_NA_0
0,C_ID_00007093c1,1.0,1.0,2.0,0.0,0.00,0.0,0.0,2.0,0.000000,...,0,151,0.006623,0.0,1,150,0.185430,0.0,28,123
1,C_ID_0001238066,1.0,1.0,25.0,0.0,0.08,0.0,2.0,23.0,0.000000,...,0,144,0.152778,0.0,22,122,0.062500,0.0,9,135
2,C_ID_0001506ef0,1.0,1.0,1.0,0.0,0.00,0.0,0.0,1.0,0.000000,...,0,67,0.000000,0.0,0,67,0.000000,0.0,0,67
3,C_ID_0001793786,1.0,1.0,31.0,0.0,0.00,0.0,0.0,31.0,0.258065,...,0,247,0.004049,0.0,1,246,0.469636,0.0,116,131
4,C_ID_000183fdda,1.0,1.0,11.0,0.0,0.00,0.0,0.0,11.0,0.000000,...,0,150,0.006667,0.0,1,149,0.026667,0.0,4,146


In [98]:
# Merge our two types of aggregations and consolidate

bool_features = pd.merge(bool_features, merged, on='card_id', how='outer')

for col in bool_features.columns:

    unique_values = bool_features[col].unique()

    if set(unique_values) == {0.0, 1.0}:
        bool_features[col] = bool_features[col].astype(bool)
    elif bool_features[col].dtype == 'int64':
        bool_features[col] = bool_features[col].astype('int32')
    elif bool_features[col].dtype == 'float64':
        bool_features[col] = bool_features[col].astype('float32')

bool_features.shape

(325540, 761)

In [101]:
# Check memory usage

bool_features.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325540 entries, 0 to 325539
Columns: 761 entries, card_id to c2_NA_sum0_c3_NA_0
dtypes: bool(11), float32(660), int32(89), object(1)
memory usage: 955.9 MB


In [103]:
# Save features to .csv

bool_features.to_csv('./Data/bool_features.csv', index=False)

# Aggregating Numerical Columns

In [105]:
# Isolate relevant features, we will still make use of booleans

num_cols = [
    'installments', 'month_lag', 'purchase_amount', 'year', 'month', 'day', 'hour'
]

num_df = df[['card_id'] + num_cols + list(renaming.keys())]
num_df.rename(columns=renaming, inplace=True)
num_df.head()

,card_id,installments,month_lag,purchase_amount,year,month,day,hour,new_flag,auth_flag,c1,c2_2,c2_3,c2_4,c2_5,c2_NA,c3_B,c3_A,c3_NA
7258152,C_ID_da2090f28e,0,-11,-0.686802,2017,1,1,0,False,True,False,False,False,False,False,False,False,False,False
18429662,C_ID_efced389a0,1,-12,-0.566590,2017,1,1,0,False,True,False,False,True,False,False,False,True,False,False
14876034,C_ID_83561fe74a,1,-13,-0.559227,2017,1,1,0,False,True,False,False,False,False,False,False,True,False,False
28523797,C_ID_479fd6392a,1,-1,-0.737892,2017,1,1,0,False,True,True,False,False,False,False,True,True,False,False
19914410,C_ID_1cf6056088,0,-4,0.004418,2017,1,1,0,False,True,False,False,False,False,False,False,False,False,False


In [111]:
# More aggregation functions

def range(series):
    return series.max() - series.min()

In [112]:
# Aggregating by card_id

num_features = num_df.groupby('card_id')[num_cols].agg([
    'mean',
    'median',
    'sum',
    mode,
    'std',
    'skew',
    'min',
    'max',
    range,
    'var'
])

num_features.columns = [f"{col}_{agg}" for col, agg in num_features.columns]
num_features.reset_index(inplace=True)

num_features = num_features.astype({
    col: 'int32' if num_features[col].dtype == 'int64' else num_features[col].dtype
    for col in num_features.columns
})

num_features.head()

,installments_mean,installments_median,installments_sum,installments_mode,installments_std,installments_skew,installments_min,installments_max,installments_range,installments_var,...,hour_mean,hour_median,hour_sum,hour_mode,hour_std,hour_skew,hour_min,hour_max,hour_range,hour_var
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,1.284768,1.0,194,1,0.760504,3.339258,1,6,5,0.578366,...,14.403974,15.0,2175,16,4.209796,-0.513693,0,22,22,17.722384
C_ID_0001238066,1.614865,1.0,239,1,1.601545,3.036936,-1,10,11,2.564948,...,14.844595,16.0,2197,19,5.873627,-0.939797,0,23,23,34.499494
C_ID_0001506ef0,0.014925,0.0,1,0,0.122169,8.185353,0,1,1,0.014925,...,12.552239,12.0,841,12,3.163636,-0.609968,0,21,21,10.008593
C_ID_0001793786,0.020243,0.0,5,0,0.141116,6.854971,0,1,1,0.019914,...,15.080972,15.0,3725,13,5.103156,-1.059279,0,23,23,26.042197
C_ID_000183fdda,1.806452,1.0,280,1,2.070500,2.681747,-1,10,11,4.286971,...,16.393548,18.0,2541,22,5.408623,-1.125058,0,23,23,29.253205


In [133]:
# Fix one boolean at a time to either 0 or 1 and run statistics on all the numerical features

def num_agg(num_df, num_cols, bool_cols):

    merged_features = None
    steps = len(bool_cols)
    start_time = time.time()

    for i, group in enumerate(bool_cols):

        features = num_df.groupby(['card_id', group])[num_cols].agg([
            'mean',
            'median',
            'sum',
            mode,
            'std',
            'skew',
            'min',
            'max',
            range,
            'var'
        ]).reset_index()

        features.columns = ['card_id', group] + [f"{col}_{agg}_{group}" for col, agg in features.columns[2:]]

        features = features.astype({
            col: 'int32' if features[col].dtype == 'int64' else features[col].dtype
            for col in features.columns
        })

        features1 = features[features[group].to_numpy()]
        features0 = features[~features[group].to_numpy()]
        features1.drop(group, axis=1, inplace=True)
        features0.drop(group, axis=1, inplace=True)
        features1.columns = ['card_id'] + [f"{col}_1" for col in features1.columns[1:]]
        features0.columns = ['card_id'] + [f"{col}_0" for col in features0.columns[1:]]

        features = pd.merge(features1, features0, on='card_id', how='outer')

        if merged_features is None:
            merged_features = features
        else:
            merged_features = pd.merge(merged_features, features, on='card_id')

        elapsed_time = time.time() - start_time
        print(f"[{i+1}/{steps}]\tshape:", merged_features.shape, "\ttime:", elapsed_time)

    return merged_features

In [134]:
# Aggregate using above function

num_merged = num_agg(num_df, num_cols, bool_cols)
num_merged.head()

[1/11]	shape: (325540, 141) 	time: 450.485467672348
[2/11]	shape: (325540, 281) 	time: 899.884345293045
[3/11]	shape: (325540, 421) 	time: 1253.3202724456787
[4/11]	shape: (325540, 561) 	time: 1533.6162810325623
[5/11]	shape: (325540, 701) 	time: 1840.314953327179
[6/11]	shape: (325540, 841) 	time: 2125.7898766994476
[7/11]	shape: (325540, 981) 	time: 2451.9139795303345
[8/11]	shape: (325540, 1121) 	time: 2827.1897642612457
[9/11]	shape: (325540, 1261) 	time: 3243.872863292694
[10/11]	shape: (325540, 1401) 	time: 3612.9010438919067
[11/11]	shape: (325540, 1541) 	time: 3926.4364240169525


,card_id,installments_mean_new_flag_1,installments_median_new_flag_1,installments_sum_new_flag_1,installments_mode_new_flag_1,installments_std_new_flag_1,installments_skew_new_flag_1,installments_min_new_flag_1,installments_max_new_flag_1,installments_range_new_flag_1,...,hour_mean_c3_NA_0,hour_median_c3_NA_0,hour_sum_c3_NA_0,hour_mode_c3_NA_0,hour_std_c3_NA_0,hour_skew_c3_NA_0,hour_min_c3_NA_0,hour_max_c3_NA_0,hour_range_c3_NA_0,hour_var_c3_NA_0
0,C_ID_00007093c1,1.000000,1.0,2.0,1.0,0.000000,NaN,1.0,1.0,0.0,...,14.403974,15.0,2175,16,4.209796,-0.513693,0,22,22,17.722384
1,C_ID_0001238066,1.640000,1.0,41.0,1.0,2.118962,3.087259,-1.0,10.0,11.0,...,14.881944,16.0,2143,19,5.933780,-0.954651,0,23,23,35.209742
2,C_ID_0001506ef0,0.000000,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,...,12.552239,12.0,841,12,3.163636,-0.609968,0,21,21,10.008593
3,C_ID_0001793786,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,15.080972,15.0,3725,13,5.103156,-1.059279,0,23,23,26.042197
4,C_ID_000183fdda,1.454545,1.0,16.0,1.0,1.293340,0.291348,-1.0,4.0,5.0,...,16.520000,18.0,2478,22,5.438762,-1.192940,0,23,23,29.580134


In [135]:
# Merge our two types of aggregations and consolidate

num_features = pd.merge(num_features, num_merged, on='card_id', how='outer')

for col in num_features.columns:

    unique_values = num_features[col].unique()

    if set(unique_values) == {0.0, 1.0}:
        num_features[col] = num_features[col].astype(bool)
    elif num_features[col].dtype == 'int64':
        num_features[col] = num_features[col].astype('int32')
    elif num_features[col].dtype == 'float64' and "purchase" not in col:
        num_features[col] = num_features[col].astype('float32')

num_features.shape

(325540, 1611)

In [136]:
# Check memory usage

num_features.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325540 entries, 0 to 325539
Columns: 1611 entries, card_id to hour_var_c3_NA_0
dtypes: bool(4), float32(1260), float64(230), int32(116), object(1)
memory usage: 2.2 GB


In [137]:
# Save features to .csv

num_features.to_csv('./Data/num_features.csv', index=False)

# Aggregating by ID Columns

In [141]:
# Isolate relevant features, we will still make use of booleans

id_cols = [
    'city_id', 'merchant_category_id', 'merchant_id', 'state_id', 'subsector_id'
]

id_df = df[['card_id'] + id_cols + list(renaming.keys())]
id_df.rename(columns=renaming, inplace=True)
id_df.head()

,card_id,city_id,merchant_category_id,merchant_id,state_id,subsector_id,new_flag,auth_flag,c1,c2_2,c2_3,c2_4,c2_5,c2_NA,c3_B,c3_A,c3_NA
7258152,C_ID_da2090f28e,69,623,M_ID_f001319a61,9,4,False,True,False,False,False,False,False,False,False,False,False
18429662,C_ID_efced389a0,76,842,M_ID_18038b5ae7,2,37,False,True,False,False,True,False,False,False,True,False,False
14876034,C_ID_83561fe74a,233,661,M_ID_52d3026407,9,8,False,True,False,False,False,False,False,False,True,False,False
28523797,C_ID_479fd6392a,-1,839,M_ID_e5374dabc0,-1,29,False,True,True,False,False,False,False,True,True,False,False
19914410,C_ID_1cf6056088,69,278,M_ID_2cf6dc1f6f,9,37,False,True,False,False,False,False,False,False,False,False,False


In [142]:
# More aggregation functions

def modelen(series):
    return len(series.mode())

def unique(series):
    return len(series.unique())

In [148]:
# Aggregating by card_id

id_features = id_df.groupby('card_id')[id_cols].agg([
    mode,
    modelen,
    unique
])

id_features.columns = [f"{col}_{agg}" for col, agg in id_features.columns]
id_features.reset_index(inplace=True)

id_features = id_features.astype({
    col: 'int32' if id_features[col].dtype == 'int64' else id_features[col].dtype
    for col in id_features.columns
})

id_features.head()

,card_id,city_id_mode,city_id_modelen,city_id_unique,merchant_category_id_mode,merchant_category_id_modelen,merchant_category_id_unique,merchant_id_mode,merchant_id_modelen,merchant_id_unique,state_id_mode,state_id_modelen,state_id_unique,subsector_id_mode,subsector_id_modelen,subsector_id_unique
0,C_ID_00007093c1,244,1,5,307,1,19,M_ID_9400cf2342,1,31,2,1,4,19,1,13
1,C_ID_0001238066,314,1,19,307,1,34,M_ID_d17aabd756,1,90,9,1,6,19,1,19
2,C_ID_0001506ef0,137,1,3,705,1,19,M_ID_b1fc88154d,1,29,19,1,2,33,1,12
3,C_ID_0001793786,179,1,11,278,1,57,M_ID_923d57de8d,1,150,-1,1,5,37,1,25
4,C_ID_000183fdda,161,1,10,367,1,38,M_ID_f9cfe0a43b,1,84,3,1,7,16,1,21


In [149]:
# Fix one boolean at a time to either 0 or 1 and run statistics on all the id features

def id_agg(id_df, id_cols, bool_cols):

    merged_features = None
    steps = len(bool_cols)
    start_time = time.time()

    for i, group in enumerate(bool_cols):

        features = id_df.groupby(['card_id', group])[id_cols].agg([
            mode,
            modelen,
            unique
        ]).reset_index()

        features.columns = ['card_id', group] + [f"{col}_{agg}_{group}" for col, agg in features.columns[2:]]

        features = features.astype({
            col: 'int32' if features[col].dtype == 'int64' else features[col].dtype
            for col in features.columns
        })

        features1 = features[features[group].to_numpy()]
        features0 = features[~features[group].to_numpy()]
        features1.drop(group, axis=1, inplace=True)
        features0.drop(group, axis=1, inplace=True)
        features1.columns = ['card_id'] + [f"{col}_1" for col in features1.columns[1:]]
        features0.columns = ['card_id'] + [f"{col}_0" for col in features0.columns[1:]]

        features = pd.merge(features1, features0, on='card_id', how='outer')

        if merged_features is None:
            merged_features = features
        else:
            merged_features = pd.merge(merged_features, features, on='card_id')

        elapsed_time = time.time() - start_time
        print(f"[{i+1}/{steps}]\tshape:", merged_features.shape, "\ttime:", elapsed_time)

    return merged_features

In [150]:
# Aggregate using above function

id_merged = id_agg(id_df, id_cols, bool_cols)
id_merged.head()

[1/11]	shape: (325540, 31) 	time: 458.8687789440155
[2/11]	shape: (325540, 61) 	time: 900.7084941864014
[3/11]	shape: (325540, 91) 	time: 1257.4173288345337
[4/11]	shape: (325540, 121) 	time: 1535.8870153427124
[5/11]	shape: (325540, 151) 	time: 1850.7451286315918
[6/11]	shape: (325540, 181) 	time: 2138.4059381484985
[7/11]	shape: (325540, 211) 	time: 2463.44172334671
[8/11]	shape: (325540, 241) 	time: 2834.83682346344
[9/11]	shape: (325540, 271) 	time: 3246.853098154068
[10/11]	shape: (325540, 301) 	time: 3613.6353969573975
[11/11]	shape: (325540, 331) 	time: 3926.5266625881195


,card_id,city_id_mode_new_flag_1,city_id_modelen_new_flag_1,city_id_unique_new_flag_1,merchant_category_id_mode_new_flag_1,merchant_category_id_modelen_new_flag_1,merchant_category_id_unique_new_flag_1,merchant_id_mode_new_flag_1,merchant_id_modelen_new_flag_1,merchant_id_unique_new_flag_1,...,merchant_category_id_unique_c3_NA_0,merchant_id_mode_c3_NA_0,merchant_id_modelen_c3_NA_0,merchant_id_unique_c3_NA_0,state_id_mode_c3_NA_0,state_id_modelen_c3_NA_0,state_id_unique_c3_NA_0,subsector_id_mode_c3_NA_0,subsector_id_modelen_c3_NA_0,subsector_id_unique_c3_NA_0
0,C_ID_00007093c1,69.0,2.0,2.0,222.0,2.0,2.0,M_ID_00a6ca8a8a,2.0,2.0,...,19,M_ID_9400cf2342,1,31,2,1,4,19,1,13
1,C_ID_0001238066,314.0,1.0,8.0,278.0,1.0,14.0,M_ID_00a6ca8a8a,25.0,25.0,...,32,M_ID_d17aabd756,1,86,9,1,5,19,1,18
2,C_ID_0001506ef0,137.0,1.0,1.0,705.0,1.0,1.0,M_ID_ab756f937e,1.0,1.0,...,19,M_ID_b1fc88154d,1,29,19,1,2,33,1,12
3,C_ID_0001793786,69.0,1.0,7.0,278.0,1.0,21.0,M_ID_0360f86430,31.0,31.0,...,57,M_ID_923d57de8d,1,150,-1,1,5,37,1,25
4,C_ID_000183fdda,161.0,1.0,2.0,367.0,1.0,9.0,M_ID_113378fe3b,11.0,11.0,...,37,M_ID_f9cfe0a43b,1,81,3,1,6,16,1,21


In [151]:
# Merge our two types of aggregations and consolidate

id_features = pd.merge(id_features, id_merged, on='card_id', how='outer')

for col in id_features.columns:

    unique_values = id_features[col].unique()

    if set(unique_values) == {0.0, 1.0}:
        id_features[col] = id_features[col].astype(bool)
    elif id_features[col].dtype == 'int64':
        id_features[col] = id_features[col].astype('int32')
    elif id_features[col].dtype == 'float64' and "purchase" not in col:
        id_features[col] = id_features[col].astype('float32')

id_features.shape

(325540, 346)

In [152]:
# Check memory usage

id_features.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325540 entries, 0 to 325539
Columns: 346 entries, card_id to subsector_id_unique_c3_NA_0
dtypes: float32(266), int32(56), object(24)
memory usage: 862.8 MB


In [153]:
# Save features to .csv

id_features.to_csv('./Data/id_features.csv', index=False)